In [1]:
from util import local_settings
from openai import OpenAI

print("First LLM API example")
print(f"✅ OpenAI Key loaded ({local_settings.OPENAI_API_KEY[0:-15]}...)")

client = OpenAI(api_key=local_settings.OPENAI_API_KEY)

sk-TE7B7pp5i1ov4pYUKiVMT3BlbkFJNb3wF
First LLM API example
✅ OpenAI Key loaded (sk-TE7B7pp5i1ov4pYUKiVMT3BlbkFJNb3wF...)


In [68]:
from reviews_utils import *
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms import OpenAI

# Load the CSV files into DataFrames
trip_df = pd.read_csv('./data/cleanTripLisbon.csv', index_col='Unnamed: 0')
customer_df = pd.read_csv('./data/customer_data.csv')

# Create reviews_df
reviews_df = pd.DataFrame(columns=['Username', 'Place', 'OverallRating', 'Review', 'Rating'])

# Remove rows with no reviews
trip_bot_df = trip_df[trip_df['ReviewsNo'] != 0]

# Excluded 'About' because the review was a copy of it
# Select top 200 rows
trip_bot_df = trip_bot_df[['Name', 'Types', 'Rating', 'ReviewsNo']].head(200).copy()

# Pandas df as agent
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), trip_bot_df, verbose=True)

# Create a list of column names containing "Rating"
rating_columns = [col for col in trip_df.columns if 'Rating' in col if col != 'Rating']

# Create dataframe to check if number of reviews does not exceed number of trips
copy_customer = customer_df[['Username', 'Planned Trips']].copy()
copy_customer['ReviewsNo'] = 0

In [48]:
from typing import List

def generate_reviews(agent, row: pd.Series, copy_customer: pd.DataFrame, rating_columns: List[str]) -> None:
    """
    Generate reviews for a given place based on the provided row.

    ----------------------
    Parameters:
     - agent: The agent responsible for generating reviews.
     - row: The row from the dataset representing a place.
     - copy_customer: A DataFrame representing customer information.
     - rating_columns: A list of column names containing rating information.
    ----------------------
    Returns:
     - None
    """
    review_question = f'\
        You are ReviewBot, an automated service for crafting authentic reviews about places to visit in Lisbon!\
        As ReviewBot, embody the voice of different types of visitors sharing experiences and recommendations.\
        Use informal language.\
        Do not mention rating, however the review can be different from the rating in the dataset.\
        If you use \' put a \ before.\
        The length of the review must be between 10 and 100 words.\
        Example dataset row:\
        - Place: Belém Tower\
        - Features: Historical landmark\
        - Rating: 4.5/5\
        Task: Write a review for the place with index {row.index[0]} using text only.'
    
    # Check if row has 3 or less reviews
    if row['ReviewsNo'] <= 3:
        # For loop for the number or reviews
        for i in range(row['ReviewsNo']):
            # Chose random customer
            customer = customer_df.sample(n=1)
            # Get reponse
            response = agent.run(review_question)
            # Create dictionary with new information
            review_dict = {
                'Username': customer['Username'],
                'Place': row['Place'],
                'OverallRating': row['Rating'],
                'Review': response,
                'Rating': get_rating(row, rating_columns)
            }
            # Make the dictionary a row of reviews_df
            reviews_df.append(review_dict, ignore_index=True)
            # Count if the number of reviews does not exceed the number of trips for the customer
            copy_customer['ReviewsNo'].iloc[customer.index[0]] += 1
    else:
        # For loop to create 3 reviews
        for i in range(3):
            # Chose random customer
            customer = customer_df.sample(n=1)
            # Get reponse
            response = agent.run(review_question)
            # Create dictionary with new information
            review_dict = {
                'Username': customer['Username'],
                'Place': row['Place'],
                'OverallRating': row['Rating'],
                'Review': response,
                'Rating': get_rating(row, rating_columns)
            }
            # Make the dictionary a row of reviews_df
            reviews_df.append(review_dict, ignore_index=True)
            # Count if the number of reviews does not exceed the number of trips for the customer
            copy_customer['ReviewsNo'].iloc[customer.index[0]] += 1

In [69]:
trip_bot_df = trip_bot_df[['Name', 'Types', 'Rating']].head(1).copy()

In [70]:
# Apply the function to each row in the DataFrame
trip_bot_df.apply(lambda row: generate_reviews(agent, row, copy_customer, rating_columns), axis = 1)

TypeError: 'AgentExecutor' object is not subscriptable